In [ ]:
#
# Revisar manualmente as imagens em C:\mestrado\datasets\samples_font  ok
#
#
# Pegar todas as imagens em final-32K-8lines do tipo sync_**.jpg e sobrescrever
# usando como texto a sequencia no id correspondente em portugues.
#
# Para os fontes carregar a lista de fontes cujas imagens existem em samples_font
# buscando as fontes em C:/fonts, mais as fontes do windows usado anteriormente
# Usar as fontes aleatoriamente...
#


In [ ]:
# import tensorflow as tf
# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#   raise SystemError('GPU device not found')
# print('Found GPU at: {}'.format(device_name))

# You'll generate plots of attention in order to see which parts of an image
# our model focuses on during captioning
import matplotlib.pyplot as plt

# Scikit-learn includes many helpful utilities
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import re
import numpy as np  
import os
import time
import json
from glob import glob
from PIL import Image
import pickle
import cv2
from sklearn.utils import shuffle

# print(tf.__version__) 

In [3]:
from IPython.display import clear_output, display
import os.path
from os import path
import shutil
import PIL
from pathlib import Path
from matplotlib import pyplot as plt

In [4]:
from IPython.display import clear_output

In [5]:
import random

In [6]:
import re

In [19]:
#
# gera imagem com jogadas escritas com fonte manuscrito
#
# -*- coding: utf-8 -*-
from PIL import ImageFont
from PIL import Image, ImageDraw
from PIL import ImageColor

In [7]:
def read_label( path):
  f= open( path)
  d= f.read()
  f.close()
  return d 

def write_label( path, text):
  f= open( path, "w")
  f.write( text)
  f.close()

def show( img, name= 'noname'):
  plt.imshow( img)
  plt.title( name)
  plt.show()

def show_gimage( img, name= 'noname'):
  plt.imshow( img, cmap= 'gray')
  plt.title( name)
  plt.show()

def _load_image( path):
  # return cv2.imread( path, cv2.IMREAD_GRAYSCALE)
  return cv2.imread( path)
  
def write_image( path, img):
  cv2.imwrite( path, img)

def resize( img, scale_percent= 60, verbose=False):
  #scale_percent = 60 # percent of original size
  width = int(img.shape[1] * scale_percent / 100)
  height = int(img.shape[0] * scale_percent / 100)
  dim = (width, height)
  if verbose:
    print( "scale", scale_percent)
    print( "dim", dim)
  # resize image
  return cv2.resize(img, dim, interpolation = cv2.INTER_AREA) 

In [8]:
pgnfile= "C:/mestrado/datasets/handmade/imagens-originais/pgn_30k_pt.txt"
with open(pgnfile) as f:
    lines = [line.rstrip() for line in f]
    
lines= [re.sub("[\[].*?[\]]", "", ln) for ln in lines]
lines= [ln[8:] for ln in lines]
lines= [ln.split()[0:32] for ln in lines]

pgnfile= "C:/mestrado/datasets/handmade/imagens-originais/pgn_30k_eng.txt"
with open(pgnfile) as f:
    en_lines = [line.rstrip() for line in f]
    
en_lines= [re.sub("[\[].*?[\]]", "", ln) for ln in en_lines]
en_lines= [ln[8:] for ln in en_lines]
en_lines= [ln.split()[0:32] for ln in en_lines]

In [9]:
print( lines[0], en_lines[0])

['d4', 'f5', 'Cf3', 'g6', 'g3', 'Bg7', 'Bg2', 'Cf6', 'b4', 'c6', 'c4', 'd5', 'Db3', 'Be6', 'c5', 'Ce4', 'O-O', 'Cd7', 'Bf4', 'O-O', 'Cbd2', 'Bf7', 'De3', 'De8', 'Bh6', 'Cdf6', 'Ce5', 'Cxd2', 'Dxd2', 'Cd7', 'Bxg7', 'Rxg7'] ['d4', 'f5', 'Nf3', 'g6', 'g3', 'Bg7', 'Bg2', 'Nf6', 'b4', 'c6', 'c4', 'd5', 'Qb3', 'Be6', 'c5', 'Ne4', 'O-O', 'Nd7', 'Bf4', 'O-O', 'Nbd2', 'Bf7', 'Qe3', 'Qe8', 'Bh6', 'Ndf6', 'Ne5', 'Nxd2', 'Qxd2', 'Nd7', 'Bxg7', 'Kxg7']


In [10]:
print( len( lines), len(en_lines))

30000 30000


In [ ]:
#
# le todos os fontes
#

In [104]:
new_fonts_files= glob( 'C:/mestrado/datasets/samples_font/*.ttf.jpg')

In [97]:
print( len( new_fonts_files))

126


In [96]:
[f for f in new_fonts_files if 'beneg' in f]

[]

In [105]:
new_fonts_path= [
    os.path.join( 'C:/mestrado/datasets/fonts', 
                 Path( f).name.replace( '.jpg', '')) for f in new_fonts_files]

In [20]:
ImageFont.truetype(new_fonts_path[0], 30)

In [106]:
new_fonts= [ImageFont.truetype(f, 30) for f in new_fonts_path]

In [107]:
fonts= [
    ImageFont.truetype('c:/Windows/Fonts/BRADHITC.TTF', 30),
    ImageFont.truetype('c:/Windows/Fonts/ITCKRIST.TTF', 30),
    ImageFont.truetype('c:/Windows/Fonts/LHANDW.TTF', 30),
    ImageFont.truetype('c:/Windows/Fonts/Inkfree.TTF', 30),
    ImageFont.truetype('c:/Windows/Fonts/OCRAEXT.TTF', 30),
    ImageFont.truetype('c:/Windows/Fonts/TEMPSITC.TTF', 30),
    ImageFont.truetype('c:/Windows/Fonts/PAPYRUS.TTF', 30),
    ImageFont.truetype('c:/Windows/Fonts/MAIAN.TTF', 30)
] 

In [108]:
all_fonts= new_fonts + fonts

In [109]:
print( len( new_fonts), len( fonts), len( all_fonts))

105 8 113


In [82]:
Path( new_fonts_path[0]).name

'A Sensible Armadillo.ttf'

In [56]:
#
# le todos os arquivos cuja imagem vai sobrescrever
#
label_files= glob(
    os.path.join( 'C:/mestrado/datasets/unified/final-32K-8lines--v2',
                 'train/labels/syn*.pgn'))

label_files.extend( glob(  
    os.path.join( 'C:/mestrado/datasets/unified/final-32K-8lines--v2',
                 'valid/labels/syn*.pgn')))

In [57]:
len( label_files)

25641

In [50]:
#
# recupera o label original de cada arquivo
#

6055

In [60]:
int( Path( label_files[0]).name[4:9])

49

In [80]:
def labelof( f):
    id= int( Path( f).name[4:9])
#     print( id)
    return lines[ id-1]

In [70]:
i= 100
print( 'labels of ', label_files[i], ' '.join( labelof( label_files[i])[0:16]))

995
labels of  C:/mestrado/datasets/unified/final-32K-8lines--v2\train/labels\syn_00995.pgn e4 e6 d4 d5 Cd2 Cf6 e5 Cfd7 Bd3 c5 c3 Cc6 Ce2 cxd4 cxd4 f6


In [ ]:
# # abre o arquivo original
# img = Image.open("C:/mestrado/datasets/syn/template/template.jpg")  # load base image
# dctx = ImageDraw.Draw(img)  # create drawing context

In [ ]:
# show( img)

In [ ]:
# fnt= ImageFont.truetype('c:/fonts/GOODDP__.TTF', 30)

In [ ]:

# def load_fonts():
#     files= glob( "C:/fonts/*.ttf")
#     fonts= []
#     for f in files:
#         fnt= ImageFont.truetype(f, 30)
#         fonts.append( (f, fnt))
#     return fonts
        
    
# all_fonts= load_fonts()    
        
# for name,fnt in all_fonts:    
#     img = Image.open("C:/mestrado/datasets/syn/template/template.jpg")  # load base image
#     dctx = ImageDraw.Draw(img)  # create drawing context

#     line= "CBxd4+"

#     dh_top= 7
#     dh_bottom= 13
#     h= (img.height - dh_top - dh_bottom)/25.0
#     w= img.width/4.0

#     txtsz = dctx.textsize(line, fnt)
#     x= (w - txtsz[0])/2
#     y= 7 + (h - txtsz[1])/2 #+ margin_top
#     dctx.text( (x, y), line, font=fnt, fill="#000000")

#     imgc= img.crop( (0, 0, w, h+7*2))
    
#     imgfile= os.path.join( "C:/mestrado/datasets/samples_font", Path(name).name+".jpg")
#     print( 'save to ', imgfile)
#     imgc.save( imgfile)    


In [ ]:
# showbig( img)

In [ ]:
def showbig( img, name= 'noname'):
    plt.figure(figsize = (10,10))    
    plt.imshow( img)
    plt.title( name)
    plt.show()
    

In [ ]:
# import glob

In [ ]:
# #
# #  gerar somente os que nao tem no handwritten
# #

# # carrega os jogos existentes
# handfiles= glob.glob( "C:/mestrado/datasets/unified/parts/handwritten/labels/*.pgn")

In [ ]:
# print( len( handfiles))

In [ ]:
# fname= handfiles[0]


In [ ]:
# Path(fname).name.split('_')

In [ ]:
# # a3__d5__d4__Nf6_5742_1494_R_part3_scan0145
# id= int( Path(fname).name.split('_')[-5])-1
# print( id)

In [ ]:
# print( read_label( fname))

In [ ]:
# " ".join( en_lines[ id])

In [ ]:
# for f in handfiles:
#     if 'scan' not in Path(f).name:
#         print( Path(f).name)


In [ ]:
# s= set()
# for f in handfiles:
#     s.add( int( Path(f).name.split('_')[-5][-4:])-1)

In [ ]:
# print( len( s))

In [ ]:
# #
# # filtra os que tem manual, os que tem tamanho menor de 16
# #
# new_lines= []
# new_en_lines=[]
# new_no= []

# for i in range( 0, len( lines)):
#     if len( en_lines[i])< 32:
#         continue
#     if i in s:
#         continue
#     new_lines.append( lines[i]) 
#     new_en_lines.append( en_lines[i]) 
#     new_no.append( i)    

In [ ]:
# print( len( new_lines))
# print( len( s))
# print( len( lines))

In [ ]:
# print( new_no[0:10])

In [ ]:
# #
# # separa em valid e test
# #
# n= len( new_lines)

# # precisa fazer shuffle?

# n_train= int(n*0.8)
# new_train_lines= new_lines[:n_train]
# new_train_en_lines=new_en_lines[:n_train]
# new_train_no= new_no[:n_train]

# new_valid_lines= new_lines[n_train:]
# new_valid_en_lines=new_en_lines[n_train:]
# new_valid_no= new_no[n_train:]

In [ ]:
# print( n, n_train)

In [85]:
len( all_fonts)

138

In [111]:
img = Image.open("C:/mestrado/datasets/syn/template/template.jpg")  # load base image
dctx = ImageDraw.Draw(img)  # create drawing context
dh_top= 7
dh_bottom= 13
h= (img.height - dh_top - dh_bottom)/25.0
w= img.width/4.0

margin_left= w/4
margin_top= h/6

def resize_fixed( img):
    _dd = (271, 230) #width, height
    return cv2.resize(img, _dd, interpolation = cv2.INTER_AREA) 

def generate_for( line, fnt=None):
#     if fnt is None:
#         fnt= random.choice( fonts)
#     print( fnt.getname())
    
    img = Image.open("C:/mestrado/datasets/syn/template/template.jpg")  # load base image
    dctx = ImageDraw.Draw(img)  # create drawing context

    i= 0
    for ln in range( 0, 8):
        for col in range( 0, 2):
            if i>= len( line):
                del dctx
                return img
            
            txtsz = dctx.textsize(line[i], fnt)

            x= w* col + (w - txtsz[0])/2
            y= dh_top + h*ln + (h - txtsz[1])/2 #+ margin_top

            dctx.text( (x, y), line[i], font=fnt, fill="#000000")
            i= i+1
            
    del dctx
    return img

def cut( img):
    h= 300
    w= 345

    img1= img[:h, :w]
    return img1

def generate_one( no, label, img_file): #pt_line, pgn, no, folder):
    id_font= no%len( all_fonts)
    img= generate_for( label, all_fonts[id_font])
    img= cut( np.asarray(img))
    img= resize_fixed( img)
    img= PIL.Image.fromarray(np.uint8(img))
    img.save( img_file)
#     img.save( img_file.replace( '.jpg', '-'+Path( new_fonts_path[id_font]).name+'.jpg'))
    return img

# img__= generate_one( 
#     new_train_lines[0], 
#     " ".join( new_train_en_lines[0]), 
#     new_train_no[0]+1, 'train')

# show( img__)
#
#  generate all
#
_files= label_files
print( 'Gerando ', len( _files))
for i in range( 0, len( _files)):
    
    if i%100== 0:
        print( i)
    
    img_file= _files[i].replace( 'labels', 'images.new').replace( '.pgn', '.jpg') 
    img__= generate_one( i, labelof( _files[i])[0:16], img_file)
#     print( img_file)
#     show( img__)

Gerando  25641
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200

In [ ]:
# for i in range( 0, len( new_train_lines)):
#     if i%100== 0:
#         print( 'train', i)
#     generate_one( new_train_lines[i], " ".join( new_train_en_lines[i]), new_train_no[i]+1, 'train')
    
# for i in range( 0, len( new_valid_lines)):
#     if i%100== 0:
#         print( 'valid', i)
#     generate_one( new_valid_lines[i], " ".join( new_valid_en_lines[i]), new_valid_no[i]+1, 'valid')